# A data tale of two cities that will make you a better Airbnb host
### Airbnb review rating analysis of the Seattle and Boston datasets from 2016/17 for Udacity Data Science Nanodegree Blog Post Task

# Intro

Reviews are of enormous importance in online marketplaces, nowhere more so than for Airbnb hosts.  Guests are potentially taking a greater risk than if they booked into a well known hotel chain, so the rating for an Airbnb property can give comfort or scare away guests in equal measure.

What actionable steps can hosts take to make sure they are maximising their overall review scores and avoiding the simple mistakes and oversights that can lead to damaging negative reviews?  And, is this the same story whether the property is in Seattle or Boston?

This analysis dives deep into the datasets of both cities to shed light on this important question.  Spoiler alert - there are some very simple steps that can be taken that might make a big improvement in overall review scores.  They might be obvious to those who already do them right, but there are a lot of hosts who would do well to take note and make a few simple changes for some great improvements in their ratings.

# Context

A key overall measure for review ratings is 'review_scores_rating'.  It is a measure out of 100 and gives an overall rating score for each property listing.

Helpfully there are a number of sub-category review scores (e.g. 'value', 'cleanliness') which are measured out of 10, and this analysis looks deeper into those to see if any of them give clues for how to unlock a higher 'review_scores_rating' overall.

There were many other possible avenues of investigation in these datasets.  After some initial exploratory data analysis, this workbook focusses in on the following three business questions for Airbnb hosts.

This project has been prepared as part of my coursework for the Udacity Data Science nanodegree and has an accompanying blog post on Medium.  (Link - https://mikedurrantsheffield.medium.com/a-data-tale-of-two-cities-that-will-make-you-a-better-airbnb-host-48a154b911ee).  The data uses the Boston and Seattle Airbnb datasets from Kaggle which contain 7,403 property listings in total from 2016/17.

The data has been handled following the CRISP-DM process from understanding the business questions and understanding the data, through to communicating answers to the problems using clear visualisations and the Medium blog post linked above.



## Question 1
### What actions can a host take to achieve a better overall rating score?


## Question 2
### From the sub-category review scores, which ones have the strongest impact on overall rating score?



## Question 3
### How do the drivers of overall rating differ by city between Seattle and Boston?

# Load in the libraries and files (Gather)

In [ ]:
# Load in the Python libraries required for this analysis

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Kaggle os set up - code not necessary if not importing files direct from Kaggle

#
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# create dataframes for the six csv data tables

df_seattle_calendar = pd.read_csv(r'/kaggle/input/seattle/calendar.csv')
df_seattle_listings = pd.read_csv(r'/kaggle/input/seattle/listings.csv')
df_seattle_reviews = pd.read_csv(r'/kaggle/input/seattle/reviews.csv')
df_boston_calendar = pd.read_csv(r'/kaggle/input/boston/calendar.csv')
df_boston_listings = pd.read_csv(r'/kaggle/input/boston/listings.csv')
df_boston_reviews = pd.read_csv(r'/kaggle/input/boston/reviews.csv')


In [ ]:
# check the shape of the listings df and see if the two cities match up

df_seattle_listings.shape

In [ ]:
df_boston_listings.shape # 3 additional columns in Boston

In [ ]:
set(df_boston_listings.columns) - set(df_seattle_listings.columns)  # these are the three extra columns

In [ ]:
set(df_seattle_listings.columns) - set(df_boston_listings.columns)  # no extra columns in Seattle

In [ ]:
# add city column to the listings to differentiate
df_boston_listings['city2'] = 'BOSTON'  
df_seattle_listings['city2'] = 'SEATTLE'  

In [ ]:
# concatenate listings tables
df_list = pd.concat([df_seattle_listings, df_boston_listings])

In [ ]:
df_list.shape # new df concat includes all 96 columns from Boston; the 3 'Boston-only' columns will not be useful in city v city comparisons

In [ ]:
# add city column to the calendar to differentiate
df_boston_calendar['city'] = 'BOSTON'  
df_seattle_calendar['city'] = 'SEATTLE'  

In [ ]:
# concatenate calendar tables
df_cal = pd.concat([df_seattle_calendar, df_boston_calendar])
df_cal

In [ ]:
df_boston_reviews.shape

In [ ]:
df_seattle_reviews.shape

In [ ]:
# add city column to the reviews tables to differentiate
df_boston_reviews['city'] = 'BOSTON'
df_seattle_reviews['city'] = 'SEATTLE'

In [ ]:
# concatenate review tables
df_revs = pd.concat([df_seattle_reviews, df_boston_reviews])
df_revs

# Begin Exploratory Data Analysis (Assess)


## CALENDAR table

In [ ]:
# calendar table is a list of availability and price data
df_cal 

### Possible business questions emerging part 1 (for long list; to be narrowed later):

* What factors drive the popularity of a property - as indicated by % fully booked?
* What factors affect the price that a property is listed at - based on listing prices (either including the date to bring in seasonality as a factor; or using a fixed date period average as the response variable to avoid seasonality)?
* What dates are particularly popular for this district/city (to bring in Boston data as well to join and compare) in terms of booking availability and price advertised?

In [ ]:
df_cal.shape   # it has just over 2.7 million rows and 5 columns

In [ ]:
df_cal[df_cal['city'] == 'BOSTON'].shape  # 1.3 million rows for Boston

In [ ]:
df_cal[df_cal['city'] == 'SEATTLE'].shape  # 1.4 million rows for Seattle

In [ ]:
df_cal.listing_id.nunique()  # it covers 7,403 unique listing id's so 7,403 unique rental properties

In [ ]:
print(df_cal.date.max())  # the calendar starts on 4th Jan 2016
print(df_cal.date.min())  # the latest date listed is 5th Sep 2017

In [ ]:
print(df_cal[df_cal['city'] == 'SEATTLE'].date.min())
print(df_cal[df_cal['city'] == 'SEATTLE'].date.max())
print(df_cal[df_cal['city'] == 'BOSTON'].date.min())
print(df_cal[df_cal['city'] == 'BOSTON'].date.max())

# slightly overlapping periods in the data
# both cities have approximately one year's worth
# boston data is very slightly more recent
# this should not give us any problems as the periods are similar and cover a year, but something to keep in mind in all analysis

In [ ]:
df_cal.isnull().sum()/df_cal.shape[0]    # around 42% of rows have the price missing; no missing values from other columns

In [ ]:
df_cal_avail = df_cal[df_cal['available']=='t']  # for rows where the date and listing_id are showing as available there are no missing price

df_cal_avail.isnull().sum()

In [ ]:
df_cal_unavail = df_cal[df_cal['available']=='f']  # for rows where the date and listing_id are showing as available there are 100% missing prices

df_cal_unavail.isnull().sum()/df_cal_unavail.shape[0]

### MISSING VALUES - for the calendar table this is all clear, with no imputing or adjusting required.  The table includes price for all properties on dates that are showing as available and does not include price for any dates that are showing as unavailable.

## LISTINGS table

In [ ]:
df_list   # wide table with 96 columns of information about each listing

### Possible business questions emerging part 2 (for long list; to be narrowed later):
* REVIEW_SCORES_RATING as response variable

    * What factors have the greatest impact on the review scores rating?
    * Does host response time affect review scores rating?
    * Does length of text written in description affect review scores rating?
    * Does host profile affect review score?  E.g. has_profile_pic, host_about length of text, host_verified etc.
    * Does amenities affect review score rating?  Number of amenities?  Categories of amenities?
    * Does price/cleaning fee etc. affect review score?
    * How does cancellation policy affect review score?
    * Which sub-review scores (i.e. accuracy, cleanliness, checkin etc. have greatest impact on overall review scores rating?
    * How do review scores differ between the two cities?
    
    
    
* CALENDAR_PRICE as response variable

    * What factors affect the price listed?
    * What price should a new property list at?
    * How does location affect price?
    * How do amenities affect price?
    * How do reviews or host status affect price?
    * How does price differ between the two cities?
    * Are there different drivers of price between the two cities?

In [ ]:
df_list.shape # 7,403 rows matches the 7,403 figure for unique listings in the calendar table; 96 cols

In [ ]:
df_list['state'].value_counts()   # shows that we need to use the city2 column for split of dataset

In [ ]:
df_list.columns  
# full list of 96 column names; most seem self explanatory; want to understand review_scores_rating to see if it is useful overall response variable
# remember last three columns are not present for the Seattle dataset

In [ ]:
# function to plot some histograms for EDA

def plot_hists(columnlist):
    """
    INPUT
    A list of columns that we want to see side by side histograms for
    
    OUTPUT
    Histograms on shared axes for the two cities
    """
    
    for col in columnlist:
        boston = df_list[df_list['city2'] == 'BOSTON'][col]
        seattle = df_list[df_list['city2'] == 'SEATTLE'][col]
        plt.hist([boston, seattle], label=['Boston', 'Seattle']) 
        plt.title(col)
        plt.legend()
        plt.show()
        
columnlist = ['accommodates', 'beds', 'bathrooms', 'price', 'square_feet', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness']
plot_hists(columnlist)

In [ ]:
# pivot tables for the categorical variables - impact on overall rating

pd.pivot_table(df_list, values='review_scores_rating', index='host_response_time', columns='city2', aggfunc='mean')

## REVIEWS table

In [ ]:
df_revs # text based view of the reviews left by guests

### Possible business questions emerging part 3 (for long list; to be narrowed later):
* REVIEW_SCORES_RATING as response variable

    * What keywords from the comments stand out as being linked to high and low review scores?
    * What keywords from the comments stand out as difference between the two cities?
    
    
    
* CALENDAR_PRICE as response variable

    * What keywords from the comments stand out as being linked to high and low price listings?

In [ ]:
df_revs.shape  # nearly 153 thousand rows of reviews for the 7 columns

In [ ]:
df_revs.listing_id.nunique() # based on 6,020 unique listing id's - this lines up with the number of listings that have more than zero reviews

In [ ]:
df_revs.listing_id.nunique()/df_list.shape[0]  # shows us that 81.3% of listings have at least one review

In [ ]:
df_list_with_reviews = df_list[df_list.number_of_reviews > 0]  # check how many listings have more than zero reviews
df_list_with_reviews.shape

# Framing the Business Questions

## Question 1

What actions can a host take to achieve a better overall rating score?

## Question 2

From the sub-category review scores, which ones have the strongest impact on overall rating score?

## Question 3

How do the drivers of overall rating differ by city between Seattle and Boston?

#### NB there are clearly a vast number of other questions that could be asked of this dataset.  Some of these were suggested in the EDA phase and could be explored at another time, but for this analysis we will focus on the three questions above only.

# Data Cleaning Stage (Clean)

In [ ]:
df_list.isnull() # a matrix showing which data has null or NaN values that will need removing or imputing before a model is run

In [ ]:
# understand the split of categorical and numerical variables

df_list.dtypes.value_counts()

In [ ]:
# list of datatypes

df_list.dtypes

In [ ]:
# create a list of the numerical variables columns to use later

list_num_vars = list(df_list.select_dtypes(include=['int64', 'float64']).columns)

list_num_vars

In [ ]:
# create a list of the categorical columns to use later

list_cat_vars = list(df_list.select_dtypes(include=['object']).columns)
list_cat_vars

In [ ]:
# how many rows for the response variable have errors

df_list.review_scores_rating.isnull().sum()

In [ ]:
# what proportion of the dataset does this affect

df_list.review_scores_rating.isnull().sum() / df_list.shape[0]

# nearly 20% have no figure for review_scores_rating - but these all need to be removed as the response variable must not be blank and should not be imputed

In [ ]:
# create new df list clean that removes the empty rows for response variable

df_list_clean = df_list.dropna(subset=['review_scores_rating'], axis=0)
df_list_clean

In [ ]:
print(7403-1460)
df_list_clean.shape # checking this df is shorter than the original df by 1,460 rows.  It has 5,943 rows so this is correct.

In [ ]:
# check how many nulls

pd.set_option('display.max_rows',100)
df_list_clean.isnull().sum()

In [ ]:
# check nulls by % of total (filtered where more than 30% of rows are nulls)

cols_to_drop = (df_list_clean.isnull().sum()/df_list_clean.shape[0])[df_list_clean.isnull().sum()/df_list_clean.shape[0] > 0.3]
cols_to_drop

In [ ]:
# create a list of these cols to drop

list_cols_to_drop = list(cols_to_drop.index)

In [ ]:
# drop these columns

df_list_clean = df_list_clean.drop(list_cols_to_drop, axis=1)
df_list_clean

In [ ]:
# check nulls by % of total

df_list_clean.isnull().sum() / df_list_clean.shape[0]


In [ ]:
# fill mean values to replace errors for all numerical columns

fill_mean = lambda col: col.fillna(col.mean()) if col.name in list_num_vars else col       # lambda function to apply to all num_var columns with missing values

df_list_clean = df_list_clean.apply(fill_mean)

df_list_clean

In [ ]:
# get dummies for categorical variables, but only if fewer than 50 categories

list_cat_cols_below50 = []   # blank list to capture cat cols below 50 categories

for i in df_list_clean.filter(list_cat_vars).columns:
    if len(df_list_clean[i].value_counts()) < 50:
        list_cat_cols_below50.append(i)
        
list_cat_cols_below50

df_list_clean = pd.get_dummies(df_list_clean, columns=list_cat_cols_below50, dummy_na=True)

df_list_clean


In [ ]:
# take a look at remaining null row proportions to decide whether to clean or drop

pd.set_option('display.max_rows', 300)
(df_list_clean.isnull().sum() / df_list_clean.shape[0]) [df_list_clean.isnull().sum() / df_list_clean.shape[0] > 0]



In [ ]:
# from the list of remaining nulls - these columns will be dropped as they would likely add little value to the model

cols_to_drop = ['summary', 'neighborhood_overview', 'transit', 'thumbnail_url', 'medium_url', 'xl_picture_url', 'host_about']

df_list_clean = df_list_clean.drop(cols_to_drop, axis=1)

In [ ]:
# remaining columns with nulls

df_list_clean.isnull().sum() [df_list_clean.isnull().sum() >0]

In [ ]:
# for remaining columns, which might be of use, we will fill with mode

mode_fill_cols = list((df_list_clean.isnull().sum() [df_list_clean.isnull().sum() >0]).index)

fill_mode = lambda col: col.fillna(col.mode()[0]) if col.name in mode_fill_cols else col    # lambda function to apply to all mode fill columns with missing values

df_list_clean = df_list_clean.apply(fill_mode)

df_list_clean




In [ ]:
# check if all columns are now showing zero nulls - ready for modelling

np.sum(df_list_clean.isnull().sum())

# Further analysis before modelling (Analyze)

In [ ]:
# looking at pre-cleaned df how does each factor appear to affect the response variable - superhost

pd.pivot_table(df_list, values='review_scores_rating', columns='city2', index='host_is_superhost', aggfunc='mean')

Super Host status appears to be strongly linked to higher rating - but this could be self-fulfilling (ie. to become a superhost a high rating is required).  Seattle non-superhosts score better than Boston.

In [ ]:
# looking at pre-cleaned df how does each factor appear to affect the response variable - host has profile pic

pd.pivot_table(df_list, values='review_scores_rating', columns='city2', index='host_has_profile_pic', aggfunc='mean')

Profile pic seems to be negatively related to review scores - this is unexpected and may impact the model.  Same pattern in both cities - Seattle no profile pic hosts have very high ratings.

In [ ]:
# looking at pre-cleaned df how does each factor appear to affect the response variable - host response time

pd.pivot_table(df_list, values='review_scores_rating', columns='city2', index='host_response_time', aggfunc='mean')

Not too much impact as long as response at least within a day, but longer than that seems to have a large penalty - the penalty being much greater in Seattle.

In [ ]:
# looking at pre-cleaned df how does each factor appear to affect the response variable - room type

pd.pivot_table(df_list, values='review_scores_rating', columns='city2', index='room_type', aggfunc='mean')

Shared room very slightly lower scores in both cities.  Otherwise not appearing to be a strong factor.

# Modelling (Model)


## A linear regression model will be used to see what predictions can be made for Review Rating Score and what factors drive this.  First using the data overall and then looking at each city individually.

In [ ]:
# import sklearn libraries

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [ ]:
df_list_clean.dtypes.value_counts()

In [ ]:

def run_pipeline(feature_list):
    """
    Creates a pipeline to generate r2 scores on a linear regression model, enabling iteration over multiple feature lists
    
    INPUT
    A single feature list of any length that we want to use as the X dataframe for explanatory variables
    
    OUTPUT
    A single R Squared score for the accuracy of the y_test_preds against the y_test dataset
    
    """

    # split dataframe in X and y explanatory and response variables

    X = df_list_clean.drop(columns=['review_scores_rating'], axis=1)
    y = df_list_clean['review_scores_rating']
    
    # trim X down to feature list only
    
    
    X = X[feature_list]      # for the feature_list argument passed to this function
    
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)    # split into train and test

    lm_model = LinearRegression(normalize=True)    # instantiate model

    lm_model.fit(X_train, y_train)    # fit model

    y_test_preds = lm_model.predict(X_test)    # predict using the model

    score = r2_score(y_test, y_test_preds)      # score the model
    
    return score

Explanatory note - for the building of the feature matrix below I ran through a few iterations and manually appended the best performing feature to the list.  Each iteration works out the best item for 'i' which is the next to append to the list.  This is more manual than I would like, but it helped me to work through each example and see how the feature list could be tuned by adding on item each iteration.

In [ ]:
list(df_list_clean.columns)

In [ ]:
# generate feature matrix (list of lists)

numeric_cols = df_list_clean.select_dtypes(include=['int64', 'uint8', 'float64'])   # create a df of the numeric cols
numeric_cols = numeric_cols.drop(columns=['review_scores_rating'], axis=1)      # drop the response variable from this df

num_cols_list = list(numeric_cols.columns)     # turn the df into a list of column names

feature_matrix = []         


# for loop to generate a list of lists - for each column for 'i' - to be passed to the pipeline for scoring

for i in num_cols_list[1:]:                                  
    feature_matrix.append(
                            [
                            'review_scores_value', 
                            'review_scores_cleanliness', 
                            'review_scores_checkin', 
                            'review_scores_location', 
                            'review_scores_accuracy',
                            'reviews_per_month',
                            'host_identity_verified_t',
                            'review_scores_communication',
                            'bedrooms',
                            'room_type_Entire home/apt',
                            'property_type_Loft',
                            'calendar_updated_4 months ago',
                            'calendar_updated_4 weeks ago',
                            'require_guest_phone_verification_t',
                            'calendar_updated_2 days ago',
                            'property_type_Other',
                            'calendar_updated_7 months ago',
                            'host_listings_count',
                            'host_response_time_a few days or more',
                            i,
                            ]
                        )

feature_matrix




In [ ]:
# create a dataframe ready to capture scores from the feature list and populate that df with the feature list and the score

outputs = pd.DataFrame({'feature_list':[], 'score':[]})

i=0

for n in feature_matrix:
    score = run_pipeline(n)
    outputs.loc[i] = np.array([n, score], dtype=object)
    i+=1

In [ ]:
# examine the feature list with the highest r squared score from the test

list(outputs.sort_values('score', ascending=False).head(1).feature_list)

In [ ]:
# view the scoring dataframe, sorted by highest score first

outputs.sort_values('score', ascending=False)

In [ ]:
# run model once for highest performing feature set

feature_list_best = ['review_scores_value', 
                'review_scores_cleanliness', 
                'review_scores_checkin', 
                'review_scores_location', 
                'review_scores_accuracy',
                'reviews_per_month',
                'host_identity_verified_t',
                'review_scores_communication',
                'bedrooms',
                'room_type_Entire home/apt',
                'property_type_Loft',
                'calendar_updated_4 months ago',
                'calendar_updated_4 weeks ago',
                'require_guest_phone_verification_t',
                'calendar_updated_2 days ago',
                'property_type_Other',
                'calendar_updated_7 months ago',
                'host_listings_count',
                'host_response_time_a few days or more',
               ]



# split dataframe in X and y explanatory and response variables

X_best = df_list_clean.drop(columns=['review_scores_rating'], axis=1)
y = df_list_clean['review_scores_rating']

# trim X down to feature list only


X_best = X_best[feature_list_best]      # for the feature_list argument passed to this function


X_best_train, X_best_test, y_train, y_test = train_test_split(X_best, y, test_size=.30, random_state=42)    # split into train and test

lm_model = LinearRegression(normalize=True)    # instantiate model

lm_model.fit(X_best_train, y_train)    # fit model

y_test_preds = lm_model.predict(X_best_test)    # predict using the model

score = r2_score(y_test, y_test_preds)      # score the model

print(score)





In [ ]:
# which features have a stronger weighting on the model - using coefficients?

def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_best_train)

#A quick look at the top results
coef_df.head(20)

In [ ]:
pd.pivot_table(df_list, index='review_scores_value', columns='city2', values='review_scores_rating', aggfunc='mean').sort_values('review_scores_value', ascending=False)

In [ ]:
pd.pivot_table(df_list, index='review_scores_cleanliness', columns='city2', values='review_scores_rating', aggfunc='mean').sort_values('review_scores_cleanliness', ascending=False)

In [ ]:
df_list_clean[df_list_clean.city2_BOSTON == True]

In [ ]:
# run model once for highest performing feature set - BOSTON

feature_list_best = ['review_scores_value', 
                'review_scores_cleanliness', 
                'review_scores_checkin', 
                'review_scores_location', 
                'review_scores_accuracy',
                'reviews_per_month',
                'host_identity_verified_t',
                'review_scores_communication',
                'bedrooms',
                'room_type_Entire home/apt',
                'property_type_Loft',
                'calendar_updated_4 months ago',
                'calendar_updated_4 weeks ago',
                'require_guest_phone_verification_t',
                'calendar_updated_2 days ago',
                'property_type_Other',
                'calendar_updated_7 months ago',
                'host_listings_count',
                'host_response_time_a few days or more',
               ]


# split data frame to be only Boston

df_boston = df_list_clean[df_list_clean.city2_BOSTON == True]


# split dataframe in X and y explanatory and response variables

X_boston = df_boston.drop(columns=['review_scores_rating'], axis=1)
y_boston = df_boston['review_scores_rating']

# trim X down to feature list only


X_best_boston = X_boston[feature_list_best]      # for the feature_list argument passed to this function


X_best_boston_train, X_best_boston_test, y_boston_train, y_boston_test = train_test_split(X_best_boston, y_boston, test_size=.30, random_state=42)    # split into train and test

lm_model_boston = LinearRegression(normalize=True)    # instantiate model

lm_model_boston.fit(X_best_boston_train, y_boston_train)    # fit model

y_boston_test_preds = lm_model_boston.predict(X_best_boston_test)    # predict using the model

score = r2_score(y_boston_test, y_boston_test_preds)      # score the model

print(score)



In [ ]:
# which features have a stronger weighting on the model - BOSTON - using coefficients?

#Use the function
coef_df_boston = coef_weights(lm_model_boston.coef_, X_best_boston_train)

#A quick look at the top results
coef_df_boston.head(20)

In [ ]:
# run model once for highest performing feature set - SEATTLE

feature_list_best = ['review_scores_value', 
                'review_scores_cleanliness', 
                'review_scores_checkin', 
                'review_scores_location', 
                'review_scores_accuracy',
                'reviews_per_month',
                'host_identity_verified_t',
                'review_scores_communication',
                'bedrooms',
                'room_type_Entire home/apt',
                'property_type_Loft',
                'calendar_updated_4 months ago',
                'calendar_updated_4 weeks ago',
                'require_guest_phone_verification_t',
                'calendar_updated_2 days ago',
                'property_type_Other',
                'calendar_updated_7 months ago',
                'host_listings_count',
                'host_response_time_a few days or more',
               ]


# split data frame to be only Seattle

df_seattle = df_list_clean[df_list_clean.city2_SEATTLE == True]


# split dataframe in X and y explanatory and response variables

X_seattle = df_seattle.drop(columns=['review_scores_rating'], axis=1)
y_seattle = df_seattle['review_scores_rating']

# trim X down to feature list only


X_best_seattle = X_seattle[feature_list_best]      # for the feature_list argument passed to this function


X_best_seattle_train, X_best_seattle_test, y_seattle_train, y_seattle_test = train_test_split(X_best_seattle, y_seattle, test_size=.30, random_state=42)    # split into train and test

lm_model_seattle = LinearRegression(normalize=True)    # instantiate model

lm_model_seattle.fit(X_best_seattle_train, y_seattle_train)    # fit model

y_seattle_test_preds = lm_model_seattle.predict(X_best_seattle_test)    # predict using the model

score = r2_score(y_seattle_test, y_seattle_test_preds)      # score the model

print(score)


In [ ]:
# which features have a stronger weighting on the model - SEATTLE - using coefficients?


#Use the function
coef_df_seattle = coef_weights(lm_model_seattle.coef_, X_best_seattle_train)

#A quick look at the top results
coef_df_seattle.head(20)

In [ ]:
pd.pivot_table(df_list, index='host_response_time', columns='city2', values='review_scores_rating', aggfunc='mean').sort_values('host_response_time', ascending=False)

In [ ]:
pd.pivot_table(df_list, index='review_scores_cleanliness', columns='city2', values='review_scores_rating', aggfunc='mean').sort_values('review_scores_cleanliness', ascending=False)

# Presentation outputs (Visualise)

In [ ]:
# need a range of well formatted visuals bringing out the answers to key questions...

In [ ]:
list(df_list.columns)

In [ ]:
# cat plot for responding time

sns.set_theme(style='whitegrid')

g = sns.catplot(
        data=df_list, kind='bar',
        x='host_response_time', y='review_scores_rating', hue='city2',
        order=['within an hour', 'within a few hours', 'within a day', 'a few days or more'],
        ci='sd', palette='dark', alpha=.6, height=6, aspect=1.5,
        )

g.despine(left=True)
g.set_axis_labels("", "Review_Scores_Rating (out of 100)")
g.legend.set_title("")

In [ ]:
# heatmap of correlations selected

df_list_clean_subset = df_list_clean[['review_scores_value', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_rating']]


f, ax = plt.subplots(figsize=(15,9))
sns.heatmap(df_list_clean_subset.corr(),
            annot=True,
            fmt='0.2f',
            linewidths=.6,
            ax=ax,
           )

In [ ]:
# cleanliness scores versus overall score by city

sns.set_theme(style='white')

sns.relplot(
            x=df_list['review_scores_cleanliness'],
            y=df_list['review_scores_rating'],
            hue=df_list['city2'],
            size=df_list['number_of_reviews'],
            sizes=(3,300),
            alpha=0.5,
            palette='muted',
            height=10,
            data='df_list',
            )

# Conclusions

## Question 1
### What actions can a host take to achieve a better overall rating score?
There are many things hosts can do that should improve their overall rating score.  Aside from the obvious 'get better ratings on all the sub-categories' (see Question 2) there are clear actions that can be taken that should make a difference.

#### Three very simple things that stand out from the data:
* Respond to your guests as soon as you can - certainly within one day.  For hosts responding within 'a few days or more' there is a clear penalty in the overall review scores.
* Keep your calendar up to date - this small task has a high predictive value on the overall rating score.  Leave your calendar for 4 months between updates and you are likely to be penalised in the reviews, whereas if you have updated it in the last two days it is likely that your review score will be better.  Why this is the case is uncertain, but could well be linked to your overall levels of communication and accuracy which give guests a better overall experience.
* Verify your identity and require your guests to verify their phone number.  These two items both link strongly with a higher overall review score and are both very simple housekeeping tasks to take care of.  Perhaps these are also linked to making sure the overall communication is stronger between host and guest.




## Question 2
### From the sub-category review scores, which ones have the strongest impact on overall rating score?
From the review sub-category scores, the strongest influencer of overall review scores rating seems to be for 'value'.  This suggests a host needs to get their pricing and their offering in line with what visitors expect for accommodation that they would rate as being 'great value'.  It has the highest coefficient for predicting the overall review scores rating.  

The second highest predictor in terms of sub-categories is the 'cleanliness' score. A very simple thing to get right, and clearly reflected in the overall rating for the property.

Close behind, in third position, is 'communication'.  Another simple one for hosts to take action and improve, for a better overall rating score.




## Question 3
### How do the drivers of overall rating differ by city between Seattle and Boston?
Looking at the two cities independently, the overall key drivers were mostly the same.  The same message of: 'communicate with your guests', 'keep your property (and your Airbnb profile) clean' and 'offer good value' should help drive better ratings in both cities.

A few things stand out as different between the cities:
* The penalty seems to be higher in Seattle for slow responsiveness in communication.
* For all 'cleanliness' scores below the best possible rating of 10, Boston seems to have a higher penalty with lower overall ratings than Seattle for the lower cleanliness scores.